In [1]:
import pandas as pd

import requests
try:
    import StringIO as io
except ImportError:
    import io

In [2]:
#Set the folder for APIKEY.py file
%cd "/nfshome/pmb434"

#Load API Keys
from APIKEY import USCensusKey

#Go back to previuos folder
%cd -

/nfshome/pmb434
/projects/ml_final_project/shared


In [3]:
#https://api.census.gov/data/2017/acs/acs5/variables.html
variables = {
    'B19013_001E':'median_household_income',
    'B02001_002E':'white_population',
    'B01003_001E':'total_population',
    'B09021_022E':'over_65'
}

ids = ','.join(variables.keys())

nyc_county_boro = {
    61:1, #MN
    5:2, #BX
    47:3, #BK
    81:4, #QN
    85:5 #SI
}

In [4]:
url = "https://api.census.gov/data/2017/acs/acs5/?get={}&for=tract:*&in=state:36&key={}".format(ids,USCensusKey)

resp = requests.request('GET', url).content
df = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']',''))).iloc[:,:-1]

#Only keep NYC counties
df = df.loc[df.county.apply(lambda x: x in nyc_county_boro.keys()),:].reset_index(drop=True)

#Add boro
df['boro'] = df.county.apply(lambda x: nyc_county_boro[x])

#Rename variables
df.rename(columns=variables, inplace=True)

#Missing data to Nan
df.replace({-666666666:None},inplace=True)

df.head()

,over_65,total_population,white_population,median_household_income,state,county,tract,boro
0,0,7411,1464,NaN,36,5,100,2
1,786,5058,2665,62773.0,36,5,200,2
2,707,5944,2616,82697.0,36,5,400,2
3,748,6115,2685,33118.0,36,5,1600,2
4,89,2817,519,40117.0,36,5,1900,2


In [5]:
df['over_65_pct'] = df.over_65/df.total_population
df['over_white_pct'] = df.white_population/df.total_population

df.head()

,over_65,total_population,white_population,median_household_income,state,county,tract,boro,over_65_pct,over_white_pct
0,0,7411,1464,NaN,36,5,100,2,0.000000,0.197544
1,786,5058,2665,62773.0,36,5,200,2,0.155397,0.526888
2,707,5944,2616,82697.0,36,5,400,2,0.118943,0.440108
3,748,6115,2685,33118.0,36,5,1600,2,0.122322,0.439084
4,89,2817,519,40117.0,36,5,1900,2,0.031594,0.184239


In [6]:
df.to_csv('data/ACS_nyc.csv', index=False)